In [1]:
import grequests, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth
from datetime import datetime
from time import sleep
import pytz
from pyModbusTCP.client import ModbusClient
from IPython.display import clear_output
import signal
from contextlib import contextmanager

PI_string = ['https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwdrlOxEyz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w2NjUwVlw2NjUwLVBQLTAyMg/value',
             'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwlVlFzEyz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w2NjUwVlw2NjUwLVBQLTAyMw/value',
             'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwYKic00yz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w2NjUwVlw2NjUwLVBQLTAyNA/value']

now = datetime.now()
new_timezone = pytz.timezone("Etc/GMT+4")
client_sentina3 = ModbusClient(host="192.168.168.105", port=1234)
client_sentina3.open()

True

In [2]:
nombre_archivo = 'Sentina3_{}-{}-{}.xlsx'.format(now.year,now.month,now.day)
sent3_pp22, sent3_pp23, sent3_pp24 = pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = [])
with pd.ExcelWriter('~/Escritorio/Caserones/datos/Sentina3/'+nombre_archivo) as writer:  
    sent3_pp22.to_excel(writer,sheet_name='PP-022')
    sent3_pp23.to_excel(writer,sheet_name='PP-023')
    sent3_pp24.to_excel(writer,sheet_name='PP-024')

In [3]:
@contextmanager
def timeout(time):
    # Register a function to raise a TimeoutError on the signal.
    signal.signal(signal.SIGALRM, raise_timeout)
    # Schedule the signal to be sent after ``time``.
    signal.alarm(time)

    try:
        yield
    except TimeoutError:
        pass
    finally:
        # Unregister the signal so it won't be triggered
        # if the timeout is not reached.
        signal.signal(signal.SIGALRM, signal.SIG_IGN)

def raise_timeout(signum, frame):
    raise TimeoutError
    
def pedir_datos(pi_string):
    with timeout(30):
        reqs = [grequests.get(pi_string,auth=HTTPBasicAuth('vulco.esed','vpump$mlcc2021'))]
        valores = grequests.map(reqs)
    return valores

In [ ]:
while True:    
    try:
        now = datetime.now()
        if (nombre_archivo != 'Sentina3_{}-{}-{}.xlsx'.format(now.year,now.month,now.day)):
            sent3_pp22, sent3_pp23, sent3_pp24 = pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = [])

        clear_output(wait=True)
        print("Sentina 3")   
        
        ###
        print('PP-022')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[0])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            pres_imp = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            flow = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][2]['Value']['Value'] 
            pump_run = json.loads(valores[0].text)['Items'][3]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][4]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][5]['Value']['Value']
            corr_total = json.loads(valores[0].text)['Items'][6]['Value']['Value']

            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
        
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)

            hor_logica, estado, ef_inst, velocidad_motor, corr_A, corr_B, corr_C  = 0, 0, 0, 0, 0, 0, 0
            temp_dev1A, temp_dev1B, temp_dev1C, temp_dev2A, temp_dev2B, temp_dev2C, temp_descA, temp_descB = 0, 0, 0, 0, 0, 0, 0, 0
            pres_des, velocidad_bomba, temp_bomba, acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax = 0, 0, 0, 0, 0, 0, 0, 0, 0
            p2p_x, p2p_y, p2p_z, id_sensor, bateria_sensor, rssi_sensor  = 0, 0, 0, 0, 0, 0

            pump_health = 91
            motor_health = 92
            eff_ind = 93
            operating_hours = np.int16(hor_op)
            ovall_he_pef = np.int16(np.amin([pump_health,motor_health,eff_ind]))

            print("Escribiendo al Modbus")
            client_sentina3.write_multiple_registers(1000,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                                          tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100)])
            client_sentina3.write_multiple_registers(1010,[np.int16(velocidad_motor),np.int16(corr_total), np.int16(corr_A), np.int16(corr_B),
                                                          np.int16(corr_C), np.int16(pot_act),np.int16(temp_dev1A),np.int16(temp_dev1B),
                                                          np.int16(temp_dev1C),np.int16(temp_dev2A)])
            client_sentina3.write_multiple_registers(1020,[np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB),
                                                          np.int16(pres_des), np.int16(flow), np.int16(velocidad_bomba), np.int16(temp_bomba),
                                                          np.int16(acel_hor), np.int16(acel_ver)])
            client_sentina3.write_multiple_registers(1030,[np.int16(acel_ax), np.int16(vel_hor), np.int16(vel_ver),np.int16(vel_ax),
                                                          np.int16(p2p_x), np.int16(p2p_y), np.int16(p2p_z),np.int16(id_sensor),
                                                          np.int16(bateria_sensor), np.int16(rssi_sensor)])
            client_sentina3.write_multiple_registers(1040,[pi_av, pump_health, motor_health, eff_ind, operating_hours, ovall_he_pef])

            print("Guardando dataframe")
            pp22 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],'Horas operando':[hor_op],
                                 'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],'Velocidad motor':[velocidad_motor],
                                 'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],'Corriente C':[corr_C],
                                 'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            sent3_pp22 = sent3_pp22.append(pp22,ignore_index=True)
        except:
            print('Conexion no lograda')
            client_sentina3.write_multiple_registers(1040,[0])
            pass
                
        ###
        print('PP-023')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[1])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            pres_imp = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            flow = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][2]['Value']['Value'] 
            pump_run = json.loads(valores[0].text)['Items'][3]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][4]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][5]['Value']['Value']
            corr_total = json.loads(valores[0].text)['Items'][6]['Value']['Value']

            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
        
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)

            hor_logica, estado, ef_inst, velocidad_motor, corr_A, corr_B, corr_C  = 0, 0, 0, 0, 0, 0, 0
            temp_dev1A, temp_dev1B, temp_dev1C, temp_dev2A, temp_dev2B, temp_dev2C, temp_descA, temp_descB = 0, 0, 0, 0, 0, 0, 0, 0
            pres_des, velocidad_bomba, temp_bomba, acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax = 0, 0, 0, 0, 0, 0, 0, 0, 0
            p2p_x, p2p_y, p2p_z, id_sensor, bateria_sensor, rssi_sensor  = 0, 0, 0, 0, 0, 0

            pump_health = 91
            motor_health = 92
            eff_ind = 93
            operating_hours = np.int16(hor_op)
            ovall_he_pef = np.int16(np.amin([pump_health,motor_health,eff_ind]))

            print("Escribiendo al Modbus")
            client_sentina3.write_multiple_registers(1050,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                                          tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100)])
            client_sentina3.write_multiple_registers(1060,[np.int16(velocidad_motor),np.int16(corr_total), np.int16(corr_A), np.int16(corr_B),
                                                          np.int16(corr_C), np.int16(pot_act),np.int16(temp_dev1A),np.int16(temp_dev1B),
                                                          np.int16(temp_dev1C),np.int16(temp_dev2A)])
            client_sentina3.write_multiple_registers(1070,[np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB),
                                                          np.int16(pres_des), np.int16(flow), np.int16(velocidad_bomba), np.int16(temp_bomba),
                                                          np.int16(acel_hor), np.int16(acel_ver)])
            client_sentina3.write_multiple_registers(1080,[np.int16(acel_ax), np.int16(vel_hor), np.int16(vel_ver),np.int16(vel_ax),
                                                          np.int16(p2p_x), np.int16(p2p_y), np.int16(p2p_z),np.int16(id_sensor),
                                                          np.int16(bateria_sensor), np.int16(rssi_sensor)])
            client_sentina3.write_multiple_registers(1090,[pi_av, pump_health, motor_health, eff_ind, operating_hours, ovall_he_pef])

            print("Guardando dataframe")
            pp23 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],'Horas operando':[hor_op],
                                 'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],'Velocidad motor':[velocidad_motor],
                                 'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],'Corriente C':[corr_C],
                                 'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            sent3_pp23 = sent3_pp23.append(pp23,ignore_index=True)
        except:
            print('Conexion no lograda')
            client_sentina3.write_multiple_registers(1090,[0])
            pass
        
        ###
        print('PP-024')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[2])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            pres_imp = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            flow = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][2]['Value']['Value'] 
            pump_run = json.loads(valores[0].text)['Items'][3]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][4]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][5]['Value']['Value']
            corr_total = json.loads(valores[0].text)['Items'][6]['Value']['Value']

            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
        
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)

            hor_logica, estado, ef_inst, velocidad_motor, corr_A, corr_B, corr_C  = 0, 0, 0, 0, 0, 0, 0
            temp_dev1A, temp_dev1B, temp_dev1C, temp_dev2A, temp_dev2B, temp_dev2C, temp_descA, temp_descB = 0, 0, 0, 0, 0, 0, 0, 0
            pres_des, velocidad_bomba, temp_bomba, acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax = 0, 0, 0, 0, 0, 0, 0, 0, 0
            p2p_x, p2p_y, p2p_z, id_sensor, bateria_sensor, rssi_sensor  = 0, 0, 0, 0, 0, 0

            pump_health = 91
            motor_health = 92
            eff_ind = 93
            operating_hours = np.int16(hor_op)
            ovall_he_pef = np.int16(np.amin([pump_health,motor_health,eff_ind]))

            print("Escribiendo al Modbus")
            client_sentina3.write_multiple_registers(1100,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                                          tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100)])
            client_sentina3.write_multiple_registers(1110,[np.int16(velocidad_motor),np.int16(corr_total), np.int16(corr_A), np.int16(corr_B),
                                                          np.int16(corr_C), np.int16(pot_act),np.int16(temp_dev1A),np.int16(temp_dev1B),
                                                          np.int16(temp_dev1C),np.int16(temp_dev2A)])
            client_sentina3.write_multiple_registers(1120,[np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB),
                                                          np.int16(pres_des), np.int16(flow), np.int16(velocidad_bomba), np.int16(temp_bomba),
                                                          np.int16(acel_hor), np.int16(acel_ver)])
            client_sentina3.write_multiple_registers(1130,[np.int16(acel_ax), np.int16(vel_hor), np.int16(vel_ver),np.int16(vel_ax),
                                                          np.int16(p2p_x), np.int16(p2p_y), np.int16(p2p_z),np.int16(id_sensor),
                                                          np.int16(bateria_sensor), np.int16(rssi_sensor)])
            client_sentina3.write_multiple_registers(1140,[pi_av, pump_health, motor_health, eff_ind, operating_hours, ovall_he_pef])

            print("Guardando dataframe")
            pp24 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],'Horas operando':[hor_op],
                                 'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],'Velocidad motor':[velocidad_motor],
                                 'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],'Corriente C':[corr_C],
                                 'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            sent3_pp24 = sent3_pp24.append(pp24,ignore_index=True)
        except:
            print('Conexion no lograda')
            client_sentina3.write_multiple_registers(1140,[0])
            pass
        
        ###
        print("Guardando excel")
        nombre_archivo = 'Sentina3_{}-{}-{}.xlsx'.format(now.year,now.month,now.day)
        with pd.ExcelWriter('~/Escritorio/Caserones/datos/Sentina3/'+nombre_archivo) as writer:  
            sent3_pp22.to_excel(writer,sheet_name='PP-022')
            sent3_pp23.to_excel(writer,sheet_name='PP-023')
            sent3_pp24.to_excel(writer,sheet_name='PP-024')
        sleep(5)
    except:
        now = datetime.now()
        clear_output(wait=True)
        print("Algo fallo, lo intentare de nuevo")
        fecha_error = "Timestamp: {}-{}-{} {}:{}".format(now.year,now.month,now.day,now.hour,now.minute)
        print(fecha_error)
        sleep(5)
        continue

Sentina 3
PP-022
Solicitando datos
Escribiendo al Modbus
Guardando dataframe
PP-023
Solicitando datos
Escribiendo al Modbus
Guardando dataframe
PP-024
Solicitando datos
Escribiendo al Modbus
Guardando dataframe
Guardando excel
